In [ ]:
from datetime import date
from google import genai
from google.genai import types

from pydantic import BaseModel, Field

from dotenv import load_dotenv
load_dotenv()

client = genai.Client()


class ZimbabweNationalRegistrationCard(BaseModel):
    id_number: str
    surname: str
    first_name: str
    date_of_birth: date
    village_of_origin: str | None
    place_of_birth: str
    date_of_issue: date

    class Config:
        json_encoders = {
            date: lambda v: v.strftime('%d/%m/%Y')
        }


from pydantic import BaseModel
from datetime import date
from typing import Optional

class AgentDetails(BaseModel):
    full_name_and_surname: str
    agent_number: str

class NextOfKinDetails(BaseModel):
    full_name_and_surname: str
    contact_number: str

class InsuredDetails(BaseModel):
    full_name_and_surname: str
    title: str
    id_number: str
    contact_number: str
    residential_address: str
    gender: str
    date_of_birth: date
    email_address: Optional[str]
    next_of_kin_details: NextOfKinDetails

class BankDetails(BaseModel):
    bank_name: str
    account_number: str
    branch_code: str
    date_of_birth: date  # seems unusual here, but as per image

class InsuranceDocument(BaseModel):
    agent_details: AgentDetails
    insured_details: InsuredDetails
    bank_details: BankDetails

    class Config:
        json_encoders = {
            date: lambda v: v.strftime('%d-%m-%Y')
        }

with open('images/form1.jpg', 'rb') as f:
    image_bytes = f.read()

response = client.models.generate_content(
    model='gemini-2.5-flash',
    contents=[
        types.Part.from_bytes(
            data=image_bytes,
            mime_type='image/jpeg',
        ),
        'Extract that information?'
    ],
    config={
        "response_mime_type": "application/json",
        "response_schema": InsuranceDocument,
    },
)

# print(response.text)
card_info = response.parsed
print(card_info.model_dump_json(indent=4))

### ID

In [30]:
"""
This code sample shows Prebuilt ID Document operations with the Azure AI Document Intelligence client library.
The async versions of the samples require Python 3.8 or later.

To learn more, please visit the documentation - Quickstart: Document Intelligence (formerly Form Recognizer) SDKs
https://learn.microsoft.com/azure/ai-services/document-intelligence/quickstarts/get-started-sdks-rest-api?pivots=programming-language-python
"""

from azure.core.credentials import AzureKeyCredential
from azure.ai.documentintelligence import DocumentIntelligenceClient
from azure.ai.documentintelligence.models import AnalyzeDocumentRequest, DocumentAnalysisFeature

from dotenv import load_dotenv
import os
import base64

load_dotenv()


endpoint = os.getenv("AZURE_ENDPOINT")
key = os.getenv("AZURE_KEY")

# sample document
formUrl = "https://raw.githubusercontent.com/Azure-Samples/cognitive-services-REST-api-samples/master/curl/form-recognizer/DriverLicense.png"

document_intelligence_client  = DocumentIntelligenceClient(
    endpoint=endpoint, credential=AzureKeyCredential(key)
)

# poller = document_intelligence_client.begin_analyze_document(
#     "prebuilt-idDocument", AnalyzeDocumentRequest(url_source=formUrl)
# )

local_image_path = r'D:\D_Documents\Dataal Africa\Ops Code\InsureFlow\Backend\images\id5.jpg'
with open(local_image_path, "rb") as f:
        image_data = f.read()
        base64_image = base64.b64encode(image_data).decode("utf-8")

poller = document_intelligence_client.begin_analyze_document(
    "prebuilt-idDocument",
    body={"base64Source": base64_image},
    features=[DocumentAnalysisFeature.QUERY_FIELDS],    # Specify which add-on capabilities to enable.
    query_fields=["PlaceOfBirth", "VillageOfOrigin"]
)


In [31]:
id_documents = poller.result()

for idx, id_document in enumerate(id_documents.documents):
    print("--------Recognizing ID document #{}--------".format(idx + 1))
    first_name = id_document.fields.get("FirstName")
    if first_name:
        print(
            "First Name: {} has confidence: {}".format(
                first_name.value_string, first_name.confidence
            )
        )
    last_name = id_document.fields.get("LastName")
    if last_name:
        print(
            "Last Name: {} has confidence: {}".format(
                last_name.value_string, last_name.confidence
            )
        )
    document_number = id_document.fields.get("DocumentNumber")
    if document_number:
        print(
            "Document Number: {} has confidence: {}".format(
                document_number.value_string, document_number.confidence
            )
        )
    dob = id_document.fields.get("DateOfBirth")
    if dob:
        print(
            "Date of Birth: {} has confidence: {}".format(dob.value_date, dob.confidence)
        )
    doe = id_document.fields.get("DateOfExpiration")
    if doe:
        print(
            "Date of Expiration: {} has confidence: {}".format(
                doe.value_date, doe.confidence
            )
        )
    pob = id_document.fields.get("PlaceOfBirth")
    if pob:
        print("Place Of Birth: {} has confidence: {}".format(pob.content, pob.confidence))
    
    voo = id_document.fields.get("VillageOfOrigin")
    if voo:
        print(
            "Village Of Origin: {} has confidence: {}".format(
                voo.value_string, voo.confidence
            )
        )


--------Recognizing ID document #1--------
First Name: CHIMUNDEGE EXPETROLLECADOLATOR has confidence: 0.179
Last Name: CHIMUNDEGE has confidence: 0.541
Document Number: 45- 190221 E 45 has confidence: 0.503
Date of Birth: 1994-04-03 has confidence: 0.584
Place Of Birth: KAJOKOTO has confidence: 0.443
Village Of Origin: MOUNT DARWIN has confidence: 0.981


In [14]:
result = poller.result()
for page in result.pages:
        print(f"----Analyzing layout from page #{page.page_number}----")
        print(
            f"Page has width: {page.width} and height: {page.height}, measured with unit: {page.unit}"
        )

----Analyzing layout from page #1----
Page has width: 296.0 and height: 170.0, measured with unit: LengthUnit.PIXEL


ID 2

In [ ]:
from azure.core.credentials import AzureKeyCredential
from azure.ai.documentintelligence import DocumentIntelligenceClient
from azure.ai.documentintelligence.models import AnalyzeDocumentRequest, DocumentAnalysisFeature

from dotenv import load_dotenv
import os
import base64

load_dotenv()


endpoint = os.getenv("AZURE_ENDPOINT")
key = os.getenv("AZURE_KEY")


document_intelligence_client  = DocumentIntelligenceClient(
    endpoint=endpoint, credential=AzureKeyCredential(key)
)

# poller = document_intelligence_client.begin_analyze_document(
#     "prebuilt-idDocument", AnalyzeDocumentRequest(url_source=formUrl)
# )

local_image_path = r'D:\D_Documents\Dataal Africa\Ops Code\InsureFlow\Backend\images\id1.jpg'
with open(local_image_path, "rb") as f:
        image_data = f.read()
        base64_image = base64.b64encode(image_data).decode("utf-8")

poller = document_intelligence_client.begin_analyze_document(
    "Zimbabwe_National_ID_Extractor_v1",
    body={"base64Source": base64_image}
)


In [10]:
id_documents = poller.result()

for idx, id_document in enumerate(id_documents.documents):
    print("--------Recognizing ID document #{}--------".format(idx + 1))
    first_name = id_document.fields.get("FirstName")
    if first_name:
        print(
            "First Name: {} has confidence: {}".format(
                first_name.value_string, first_name.confidence
            )
        )
    last_name = id_document.fields.get("LastName")
    if last_name:
        print(
            "Last Name: {} has confidence: {}".format(
                last_name.value_string, last_name.confidence
            )
        )
    document_number = id_document.fields.get("IdNumber")
    if document_number:
        print(
            "Document Number: {} has confidence: {}".format(
                document_number.value_string, document_number.confidence
            )
        )
    dob = id_document.fields.get("DateOfBirth")
    if dob:
        print(
            "Date of Birth: {} has confidence: {}".format(dob.value_string, dob.confidence)
        )
    doe = id_document.fields.get("DateOfExpiration")
    if doe:
        print(
            "Date of Expiration: {} has confidence: {}".format(
                doe.value_string, doe.confidence
            )
        )
    pob = id_document.fields.get("PlaceOfBirth")
    if pob:
        print("Place Of Birth: {} has confidence: {}".format(pob.content, pob.confidence))
    
    voo = id_document.fields.get("VillageOfOrigin")
    if voo:
        print(
            "Village Of Origin: {} has confidence: {}".format(
                voo.value_string, voo.confidence
            )
        )


--------Recognizing ID document #1--------
First Name: ANDREW WILLIAM L has confidence: 0.586
Last Name: ROBERTS has confidence: 0.878
Document Number: 75- 191961 R 00 has confidence: 0.866
Date of Birth: 17/11/1959 has confidence: 0.902
Place Of Birth: HARARE has confidence: 0.882
Village Of Origin: None has confidence: 0.973


In [9]:
id_documents.documents

[{'docType': 'Zimbabwe_National_ID_Extractor_v1', 'boundingRegions': [{'pageNumber': 1, 'polygon': [0, 0, 296, 0, 296, 170, 0, 170]}], 'fields': {'IdNumber': {'type': 'string', 'valueString': '75- 191961 R 00', 'content': '75- 191961 R 00', 'boundingRegions': [{'pageNumber': 1, 'polygon': [144, 33, 222, 33, 222, 43, 144, 43]}], 'confidence': 0.866, 'spans': [{'offset': 52, 'length': 15}]}, 'LastName': {'type': 'string', 'valueString': 'ROBERTS', 'content': 'ROBERTS', 'boundingRegions': [{'pageNumber': 1, 'polygon': [144, 46, 196, 45, 196, 56, 144, 56]}], 'confidence': 0.878, 'spans': [{'offset': 82, 'length': 7}]}, 'FirstName': {'type': 'string', 'valueString': 'ANDREW WILLIAM L', 'content': 'ANDREW WILLIAM L', 'boundingRegions': [{'pageNumber': 1, 'polygon': [144, 58, 248, 58, 248, 69, 144, 69]}], 'confidence': 0.586, 'spans': [{'offset': 101, 'length': 16}]}, 'DateOfBirth': {'type': 'string', 'valueString': '17/11/1959', 'content': '17/11/1959', 'boundingRegions': [{'pageNumber': 1, 

## Claim Form

In [35]:
from PIL import Image

# Load images
local_image_path_11 = r'D:\D_Documents\Dataal Africa\Ops Code\InsureFlow\Experiments\images\form\f11.jpg'
local_image_path_12 = r'D:\D_Documents\Dataal Africa\Ops Code\InsureFlow\Experiments\images\form\f12.jpg'


# Load images
image_11 = Image.open(local_image_path_11)
image_12 = Image.open(local_image_path_12)

# Save images as PDF
combined_image_path = r'D:\D_Documents\Dataal Africa\Ops Code\InsureFlow\Experiments\images\form\combined.pdf'
image_11.save(combined_image_path, "PDF", save_all=True, append_images=[image_12])

# Analyze document
with open(combined_image_path, "rb") as f:
    combined_image_data = f.read()
    base64_combined_image = base64.b64encode(combined_image_data).decode("utf-8")

poller = document_intelligence_client.begin_analyze_document(
    "Temporary_Loss_Of_Income_Protection_Proposal_Form_Extractor_v1",
    body={"base64Source": base64_combined_image}
)

In [36]:
id_documents = poller.result()

for idx, id_document in enumerate(id_documents.documents):
    print("--------Recognizing ID document #{}--------".format(idx + 1))

    # Extract fields
    agent_full_name = id_document.fields.get("AgentFullName")
    agent_number = id_document.fields.get("AgentNumber")
    full_name = id_document.fields.get("FullName")
    title = id_document.fields.get("Title")
    gender = id_document.fields.get("Gender")
    id_number = id_document.fields.get("IdNumber")
    date_of_birth = id_document.fields.get("DateOfBirth")
    contact_number = id_document.fields.get("ContactNumber")
    email = id_document.fields.get("Email")
    address = id_document.fields.get("Address")
    nok_full_name = id_document.fields.get("NOK_FullName")
    nok_contact_number = id_document.fields.get("NOK_ContactNumber")
    bank_name = id_document.fields.get("BankName")
    b_date_of_birth = id_document.fields.get("B_DateOfBirth")
    account_number = id_document.fields.get("AccountNumber")
    branch_code = id_document.fields.get("BranchCode")
    existing_insurance_with_other_company = id_document.fields.get("ExistingInsuranceWithOtherCompany")
    existing_chronic_condition = id_document.fields.get("ExistingChronicCondition")
    claim_ailment = id_document.fields.get("ClaimAilment")
    claim_amount = id_document.fields.get("ClaimAmount")
    declined_coverage = id_document.fields.get("DeclinedCoverage")
    declined_cover_reason = id_document.fields.get("DeclinedCoverReason")
    dependents = id_document.fields.get("Dependents")

    # Print extracted fields
    print("Agent Full Name: {} has confidence: {}".format(agent_full_name.value_string, agent_full_name.confidence))
    print("Agent Number: {} has confidence: {}".format(agent_number.value_string, agent_number.confidence))
    print("Full Name: {} has confidence: {}".format(full_name.value_string, full_name.confidence))
    print("Title: {} has confidence: {}".format(title.value_string, title.confidence))
    print("Gender: {} has confidence: {}".format(gender.value_string, gender.confidence))
    print("ID Number: {} has confidence: {}".format(id_number.value_string, id_number.confidence))
    print("Date of Birth: {} has confidence: {}".format(date_of_birth.value_string, date_of_birth.confidence))
    print("Contact Number: {} has confidence: {}".format(contact_number.value_string, contact_number.confidence))
    print("Email: {} has confidence: {}".format(email.value_string, email.confidence))
    print("Address: {} has confidence: {}".format(address.value_string, address.confidence))
    print("NOK Full Name: {} has confidence: {}".format(nok_full_name.value_string, nok_full_name.confidence))
    print("NOK Contact Number: {} has confidence: {}".format(nok_contact_number.value_string, nok_contact_number.confidence))
    print("Bank Name: {} has confidence: {}".format(bank_name.value_string, bank_name.confidence))
    print("Beneficiary Date of Birth: {} has confidence: {}".format(b_date_of_birth.value_string, b_date_of_birth.confidence))
    print("Account Number: {} has confidence: {}".format(account_number.value_string, account_number.confidence))
    print("Branch Code: {} has confidence: {}".format(branch_code.value_string, branch_code.confidence))
    print("Existing Insurance With Other Company: {} has confidence: {}".format(existing_insurance_with_other_company.value_string, existing_insurance_with_other_company.confidence))
    print("Existing Chronic Condition: {} has confidence: {}".format(existing_chronic_condition.value_string, existing_chronic_condition.confidence))
    print("Claim Ailment: {} has confidence: {}".format(claim_ailment.value_string, claim_ailment.confidence))
    print("Claim Amount: {} has confidence: {}".format(claim_amount.value_string, claim_amount.confidence))
    print("Declined Coverage: {} has confidence: {}".format(declined_coverage.value_selection_mark, declined_coverage.confidence))
    print("Declined Cover Reason: {} has confidence: {}".format(declined_cover_reason.value_string, declined_cover_reason.confidence))

    if dependents and dependents.value_array:
        print("Dependents:")
        for dependent in dependents.value_array:
            if dependent.value_object:
                dependent_full_name = dependent.value_object.get("FullName")
                dependent_id_number = dependent.value_object.get("IdNumber")
                dependent_date_of_birth = dependent.value_object.get("DateOfBirth")
                dependent_age = dependent.value_object.get("Age")
                dependent_gender = dependent.value_object.get("Gender")
                dependent_relationship = dependent.value_object.get("Relationship")

                if dependent_full_name:
                    print("  - Full Name: {} has confidence: {}".format(dependent_full_name.value_string, dependent_full_name.confidence))
                if dependent_id_number:
                    print("  - ID Number: {} has confidence: {}".format(dependent_id_number.value_string, dependent_id_number.confidence))
                if dependent_date_of_birth:
                    print("  - Date of Birth: {} has confidence: {}".format(dependent_date_of_birth.value_string, dependent_date_of_birth.confidence))
                if dependent_age:
                    print("  - Age: {} has confidence: {}".format(dependent_age.value_string, dependent_age.confidence))
                if dependent_gender:
                    print("  - Gender: {} has confidence: {}".format(dependent_gender.value_string, dependent_gender.confidence))
                if dependent_relationship:
                    print("  - Relationship: {} has confidence: {}".format(dependent_relationship.value_string, dependent_relationship.confidence))

--------Recognizing ID document #1--------
Agent Full Name: Tanyaradzwa Nyanyiwa has confidence: 0.688
Agent Number: 1301 has confidence: 0.976
Full Name: Kevin Bindu has confidence: 0.862
Title: Mr has confidence: 0.978
Gender: Male has confidence: 0.975
ID Number: 692033069K42 has confidence: 0.92
Date of Birth: 12/10/02 has confidence: 0.965
Contact Number: 0773 257 373 has confidence: 0.874
Email: kevin@gmail.com has confidence: 0.568
Address: 27 Mugoti Street, Zengeza 3 has confidence: 0.855
NOK Full Name: donald bindu has confidence: 0.714
NOK Contact Number: 0773 434 679 has confidence: 0.953
Bank Name: Capitec has confidence: 0.861
Beneficiary Date of Birth: 10/01/20 has confidence: 0.969
Account Number: 23741789101 has confidence: 0.971
Branch Code: 287 has confidence: 0.975
Existing Insurance With Other Company: N/A has confidence: 0.975
Existing Chronic Condition: Have Diabetes has confidence: 0.903
Claim Ailment: Diabetes has confidence: 0.935
Claim Amount: $ 3000 has confi

--------Recognizing ID document #1--------
Agent Name: John Doe has confidence: 0.543
Agent Number: 75 3379 has confidence: 0.932
Full Name: Tonderal Brandon Mutombua has confidence: 0.232
Title: Mr has confidence: 0.831
Gender: Male has confidence: 0.944
ID Number: 64-204479 has confidence: 0.832
Date of Birth: 12-10-2020 has confidence: 0.919
Contact Number: 0776 464 136 has confidence: 0.694
Email: brandon@gmail.com has confidence: 0.468
Address: 22 Alps Street, Chitungwiza has confidence: 0.538
Next of Kin Name: Luke Mutombwa has confidence: 0.44
Next of Kin Contact: 0773 798 261 has confidence: 0.876
Bank Name: CBZ has confidence: 0.455
Bank Date of Birth: 12-10-25 has confidence: 0.769
Account Number: 25 3799 has confidence: 0.834
Branch Code: 1$5 has confidence: 0.499
Existing Insurance: None has confidence: 0.483
Chronic Condition: None has confidence: 0.846
Claim Ailment: None has confidence: 0.91
Claim Amount: None has confidence: 0.874
Declined Coverage: None has confidence: